In [1]:
#Je reviens à quelque chose de beaucoup plus simple. On va prouver que l'on obtient le bon FF pour STF classique. 
#Travaillons en mode dense, facile.
# This first version is for the classical STF. 
# Now, try to get to the domain-based STF...
# Il semble que ça a marché! Donc si je comprends bien, maintenant, il serait interessant de trouver le MTF avec osrc en block preconditioning.
# Essayer de répliquer le problème de OSRC. 
# MTF validated here. Now trying to get the stf working.

import bempp.api 
import numpy as np
import mtf

from bempp.api.operators.boundary.sparse import identity
from bempp.api.assembly.blocked_operator import BlockedOperator, GeneralizedBlockedOperator

from mtf.utils import bhmie
from mtf.config import config
from mtf.functions import define_bempp_functions
from matplotlib import pyplot as plt
from decimal import Decimal

bempp.api.enable_console_logging()
M = 2

mtf.config.set_case("B")

tangential_trace, neumann_trace = define_bempp_functions(config)

k0, k1 = config["k_ext"], config["k_int"]
k0, k1 = 1, 1
lambda_par, freq = config["lambda"], config["frequency"]

polarization = config["polarization"]
direction = config["direction"]

eps_rel = config["eps_rel"]
mu_rel = config["mu_rel"]
mu0 = config["mu_ext"]
mu1 = mu_rel * mu0

eta_rel = np.sqrt(mu_rel / eps_rel)

print("The exterior wavenumber is: {0}".format(k0))
print("The interior wavenumber is: {0}".format(k1))

print("----")
print("The exterior wavelenght is: {0}".format(lambda_par))
print("The exterior frequency is: {:.2E}".format(Decimal(freq)))

# To do: Go to the multi-domain STF...

segments = [[10], [20]]
swapped_normals = [[], []]

k_int, k_ext = config["k_int"], config["k_ext"]

n = k_int / k_ext
refIndex = n
numAngles = 901
s1, s2, qext, qsca, qback, gsca = bhmie(k_ext, k_int / k_ext, numAngles)
angles = config['angles']

k_list = [k0]
eta_rel_list = [1]
mu_list = [mu0]

for index in range(M-1):
  k_list.append(k1)
  mu_list.append(mu1)
  eta_rel_list.append(eta_rel)

precision = 5

h = 2 * np.pi/(precision*k0)
grid = bempp.api.shapes.multitrace_sphere(h=h)



The exterior wavenumber is: 1
The interior wavenumber is: 1
----
The exterior wavelenght is: 1.2566370614359172
The exterior frequency is: 2.39E+8



bempp:HOST:INFO: Created grid with id 9ddce212-9839-498c-a873-6bb4057e3cd7. Elements: 56. Edges: 80. Vertices: 27


In [2]:
print(h, ': h')
print(precision, ': precision')
print(grid.number_of_edges * 2, ': N')


dA = [bempp.api.function_space(grid, "RWG", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]

p1dA = [bempp.api.function_space(grid, "DP", 1, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]

rA = [bempp.api.function_space(grid, "RWG", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]
tA = [bempp.api.function_space(grid, "SNC", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]


multitrace_ops = []
multitrace_ops_intern = []
# > Assemble all diagonal operators
for index in range(2):
  k = k_list[index]
  mu = mu_list[index]
  eta = eta_rel_list[index]
  efie = bempp.api.operators.boundary.maxwell.electric_field(dA[index], rA[index], tA[index], k)
  mfie = bempp.api.operators.boundary.maxwell.magnetic_field(dA[index], rA[index], tA[index], k)
  multitrace_ops.append(bempp.api.GeneralizedBlockedOperator([[mfie, eta * efie],[- 1/eta * efie, mfie]]))




bempp:HOST:WARNING: Setting include_boundary_dofs has no effect on this space type.
bempp:HOST:WARNING: Setting include_boundary_dofs has no effect on this space type.


1.2566370614359172 : h
5 : precision
160 : N


In [7]:
bempp.api.operators.boundary.maxwell.multitrace_operator?

Signature:
bempp.api.operators.boundary.maxwell.multitrace_operator(
    grid,
    wavenumber,
    epsilon_r=1,
    mu_r=1,
    target=None,
    space_type='magnetic_dual',
    parameters=None,
    assembler='default_nonlocal',
    device_interface=None,
    precision=None,
)
Docstring:
Simplified version of multitrace operator assembly.

Parameters
----------
grid : Grid
    Bempp grid object.
wavenumber : complex
    A real or complex wavenumber
epsilon_r : float
    Relative permittivity with respect to vacuum.
mu_r : float
    Relative permeability with respect to vacuum.
target : Grid
    The grid for the range spaces. If target is None then
    target is set to the input grid (that is the domain
    grid).
space_type : string
    One of "all_rwg", "all_bc", "magnetic_dual" (default),
    "electric_dual". These lead to the following
    choices of space, range, and dual_to_range:
    default - (RWG, RWG), (BC, BC), (SNC, SNC)
    all_dual - (BC, BC), (RWG, RWG), (RBC, RBC)
    mag

In [3]:
A0 = multitrace_ops[0].weak_form()



bempp:HOST:INFO: OpenCL CPU Device set to: pthread-AMD EPYC 7302 16-Core Processor
/usr/local/lib/python3.10/dist-packages/pyopencl/cache.py:495: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(
/usr/local/lib/python3.10/dist-packages/pyopencl/cache.py:499: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  prg.build(options_bytes, devices)


In [4]:
efie_10_20 = bempp.api.operators.boundary.maxwell.electric_field(dA[0], rA[1], tA[1], k)

In [5]:
print(A10_wf.shape)
print(A20_wf.shape)

(74, 74)
(68, 68)


In [6]:
efie_10_20.weak_form()

<34x37 DenseDiscreteBoundaryOperator with dtype=complex128>